In [5]:
import requests
import json, time
import random, datetime
from faker import Faker

with open("./mock-insert/referenten.json") as file:
    referenten = json.load(file)

with open("./mock-insert/studiengaenge.json") as file:
    studiengaenge = json.load(file)

with open("./mock-insert/studenten.json") as file:
    studenten = json.load(file)

for ref in referenten:
    requests.post("http://localhost:8080/api/evaluator", json=ref)

for studg in studiengaenge:
    requests.post("http://localhost:8080/api/study-program", json=studg)

for stud in studenten:
    requests.post("http://localhost:8080/api/student", json=stud)

# create works by combining random referent, student, study program
# 30 refs, 50 studs, ~80 stupgs

def random_iso_8601_date(years, yeare, minm, maxm):
    start_date = datetime.date(years, minm, 1)
    end_date = datetime.date(yeare, maxm, 28)
    step = datetime.timedelta(days=1)
    dates = [ start_date ]

    while start_date != end_date:
        start_date += step
        dates.append(start_date)

    return random.choice(dates).isoformat()

def random_iso_8601_datetime(years, yeare, minm, maxm, st):
    start_date = datetime.datetime(years, minm, 1)
    end_date = datetime.datetime(yeare, maxm, 28)
    step = datetime.timedelta(days=1)
    dates = [ start_date ]

    while start_date != end_date:
        start_date += step
        dates.append(start_date)

    dt = random.choice(dates)
    dt = dt.replace(hour=st, minute=0)
    return dt.isoformat()

fake = Faker()

events = []

for i in range(100):
    starttime = random.randint(8, 14)
    ref1 = random.randint(1, len(referenten))
    ref2 = random.randint(1, len(referenten))

    while ref1 == ref2:
        ref2 = random.randint(1, len(referenten))

    work = {
        "title": " ".join(fake.sentences()),
        "colloquium": random_iso_8601_datetime(2026, 2026, 2, 3, starttime),
        "colloquiumLocation": f"Room {random.randint(1, 400)}",
        "colloquiumDuration": "PT1H30M", 
        "presentationStart": f"{starttime}:00",
        "presentationEnd": f"{starttime}:30",
        "discussionStart": f"{starttime}:30",
        "discussionEnd": f"{starttime + 1}:00",
        "startDate": random_iso_8601_date(2025, 2025, 9, 10),
        "endDate": random_iso_8601_date(2026, 2026, 4, 6),
        "studentId": random.randint(1, len(studenten)),
        "studyProgramId": random.randint(1, len(studiengaenge)),
        "mainEvaluatorId": ref1,
        "mainEvaluatorWorkMark": random.randint(50, 100),
        "mainEvaluatorColloquiumMark": random.randint(50, 100),
        "secondEvaluatorId": ref2,
        "secondEvaluatorWorkMark": random.randint(50, 100),
        "secondEvaluatorColloquiumMark": random.randint(50, 100),
        "comment": " ".join(fake.sentences())
    }

    for _ in range(random.randint(1, 7)):
        event = {
            "eventType": random.choice(["PROPOSAL", "DISCUSSION", "FINAL_SUBMISSION", "REVIEW", "ARCHIVE", "ABORT"]),
            "eventDate": random_iso_8601_date(2025, 2026, 11, 1),
            "scientificWorkId": i + 1
        }

        events.append(event)

    res = requests.post("http://localhost:8080/api/scientific-work", json=work)

time.sleep(1)
for event in events:
    res = requests.post("http://localhost:8080/api/event", json=event)

